In [22]:
import os
import warnings

import pandas as pd
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
)

from datasets import Dataset

import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 16)

# Paths and Names

In [50]:
### paths and names

DATA_PATH = 'data/preprocess_v2.pickle'
MODEL_CHECKPOINT = '.log/paust_pko_t5_base_v2/checkpoint-8876'

# Model & Tokenizer

In [51]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [52]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Training Args

In [25]:
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

predict_with_generate=True
generation_max_length=128

# Functions

In [28]:
prefix = "generate keyphrases: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Inputs and Labels

In [29]:
data_df = pd.read_pickle(DATA_PATH)

In [30]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2, seed=100)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [31]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=16):   0%|          | 0/1268 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/317 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1268
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 317
})


In [32]:
tokenizer.decode(train_dataset['input_ids'][0])

'generate keyphrases: 광주시, 물류운송 드론 기술개발 공모 선정 광주시가 2백킬로그램급 화물운송용 드론 기술개발 공모사업에 선정됐습니다. 이번 사업은 오는 2025년까지 수소연료전지를 기반으로 최대 시속 백킬로미터의 속도로 탑재 중량 2백 킬로그램급 드론개발과 실증 사업 기반을 구축하는 것입니다. 광주시는 지난 1월 LIG넥스원 등과 화물용 드론 개발 업무협약을 체결했으며, 수소연료전지 기술 개발과 함께 물류 운송용 드론 시장 선점을 통한 지역 산업 성장에도 도움이 될 것으로 크게 기대하고 있습니다. 최송현</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [33]:
tokenizer.decode(train_dataset['labels'][0])

'광주시,물류운송 드론,기술개발 공모,선정,2백킬로그램급 화물운송용 드론,2025년,수소연료전지</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Trainer for Generation

In [34]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./.temp',
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
)

In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Generate

In [38]:
preds = trainer.predict(eval_dataset)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [39]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 지한솔,KLPGA 투어 E1 채리티오픈
예측: 한솔,KLPGA 투어 E1 채리티오픈,한국여자프로골프 KLPGA 투어 E1 채리티오픈,2R 선두,하민송

정답: 박명수,한수민,딸,데이트,사춘기,민서,SNS
예측: ,한수민,딸,데이트,사춘기 안왔으면,딸바보 부부,딸바보

정답: 크리스찬 루부탱,더보이즈 영훈
예측: 보이즈 영훈,프랑스 럭셔리 브랜드,크리스찬 루부탱,팝업스토어 오픈 기념 포토콜,포즈

정답: 롯데 자이언츠,5연패 탈출,KBO리그,KIA 타이거즈,17-9 승리,안치홍,딕슨 마차도
예측: ,5연패 탈출,KBO리그,KIA 타이거즈,17-9,KBO리그

정답: 프랑스 축구,선두 파리 생제르맹,복통,콘테 감독,담낭염
예측: 바페,실축,파리 생제르맹,메시,결승골,파리 생제르맹

정답: 김치 발효,유산균 발효,발효과학,풀무원,발효 김치,유산균,류코노스톡
예측: ,톡톡김치,발효 풍미,풀무원,김치,발효 풍미

정답: 카카오,태·조·이·방·원,코스피지수,코스닥지수,인터넷,삼전은 '5만전자',신고가
예측: ,코스피,코스피,코스피,코스피,코스피,코스피,코스피

정답: 이별 리콜,성유리 외모 지적
예측: ,외모 지적,이별 리콜,연인의 평가,불편한 지인,리콜녀,SBS2 예능

정답: 남다름,박서준,이른 나이에 자진 입대,정해인,유승호
예측: ,남다름,자진 입대,아역 출신 배우,자진 입대,아역 출신 배우,아역 출신 배우

정답: 신세계면세점 부산점,소외계층 청소년 장학금,장학금 지원
예측: 면세점 부산점,소외계층 청소년 장학금 지원,부산공동모금회

정답: 전유진 달성군수 후보 선거사무소 개소
예측: 유진 달성군수 후보,선거사무소 개소,더불어민주당,전유진 달성군수 후보

정답: 전용면적 84m2,첫 '20억 아파트',경기도
예측: 도,20억 아파트,매매신고,과천 중앙동,과천푸르지오써밋,전용면적 84m2

정답: K리그1,K리그2,주민규,김현욱,다이내믹 포인트,파워랭킹,경기 데이터
예측: 리그1,주민규,K리그2,김현욱,4월 K리그판 파워랭킹,김현욱

정답: 줄리어스·파타푸